In [1]:
#CONSTANTS TO RUN THE PAGE

PATH_TO_DATA="/Users/williamcochran/Code/learn_stuff/connect6/data/"
MOVES_TO_DETERMINE_GAME=10
FIRST_MOVE="J10"
BLACK=1
WHITE=-1
EMPTY=0

def swap(color):
    return color*-1



In [2]:
#Let's read some data!
import os

# File finder and reader.. returns a list of moves
def parse_game_file (filename):
    with open(filename, "r") as f:
        lines = f.readlines()
        games = []
        for line in lines:
            if line[0] != "(" and line[-1] != ")":
                print("Found a bad line: {}".format(line))
            trimmed = line[1:-2]
            keys=trimmed.split(';')
            keys = keys[3:]
            moves = [k[2:-1] for k in keys]
            if len(moves)>0:
                games.append(moves)
    return games

# Look for all files in the data path
def read_all_data (path=PATH_TO_DATA):
    games = []
    all_files = os.listdir(path)
    data_files = [file for file in all_files if file.endswith(".data")]
    for file in data_files:
        print ("Reading {}".format(file))
        games.extend(parse_game_file (PATH_TO_DATA + file))
    return games



In [3]:
# Board manipulation helper functions
import random
import copy

def make_board():
    board = []
    for a in range(19):
        b = []
        for c in range(19):
            b.append(EMPTY)
        board.append(b)
    board[9][9] = BLACK
    return board

def fill_with_random(board, stones):
    j = 2
    if len(board) != 19 or any(len(row) != 19 for row in board):
        print(f"Board at index {i} is incorrect before. Length: {len(board)}, Row lengths: {[len(row) for row in board]}")

    while stones > 0:
        x=0
        y=0
        while True:
            x = random.randint(0,18)
            y = random.randint(0,18)
            if board[x][y] == EMPTY:
                break
        
        board[x][y] = BLACK if j < 2 else WHITE
        j = (j+1) % 4
        stones = stones - 1

    if len(board) != 19 or any(len(row) != 19 for row in board):
        print(f"Board at index {i} is incorrect before. Length: {len(board)}, Row lengths: {[len(row) for row in board]}")
        raise KeyboardInterrupt()

    return board

def make_random_board(good_boards):
    board = copy.deepcopy(random.choice(good_boards))
    return fill_with_random(make_board(),random.randint(1,2))

def get_stone(move_cluster,spot):
    try:
        x = ord(move_cluster[spot]) - ord('a')
        y = ord(move_cluster[spot+1]) - ord('1')
        ans = spot + 2
        if spot+2 < len(move_cluster):
            if move_cluster[spot+2].isdigit():
                ans = ans + 1
                # Be sure to add 1 to the digit since a single digit 
                # number is assumed to start at "1" instead of "0"
                y = (y+1) * 10 + ord(move_cluster[spot+2]) - ord('1')
    except TypeError:
        print ("{} {}".format(move_cluster,spot))
        raise KeyboardInterrupt()
    return ((x,y),ans)

def count_stones (board):
    total = 0;
    for x in board:
        for y in x:
            total = total + abs(y)
    return total

def return_moves(move_cluster):
    (stone1,offset) = get_stone(move_cluster,0)
    (stone2,offset) = get_stone(move_cluster,offset)
    return (stone1,stone2)

def pretty_print(board):
    for x in board:
        line = "  "
        for y in x:
            if y == -1:
                line = line + "O "
            elif y == 1:
                line = line + "X "
            else:
                line = line + "+ "
        print(line)

def apply_move(board, stone, value):
    (x,y) = stone
    board[x][y] = value

def make_random_boards(number_to_make, good_boards):
    answer = []
    while number_to_make > 0:
        if (number_to_make % 10000) == 0:
            print ("Need to make {}".format(number_to_make))
        answer.append(make_random_board(good_boards))
        number_to_make = number_to_make - 1

    return answer


def build_board_list(game):
    board_list = []
    board = make_board()
    value = WHITE
    for move_cluster in game:
        board = copy.deepcopy(board)
        (stone1,stone2) = return_moves(move_cluster)
        apply_move(board,stone1,value)
        apply_move(board,stone2,value)
        value = swap(value)
        board_list.append(board)

    return board_list

def make_boards_for_list(game_list):
    boards = []
    for game in game_list:
        boards.extend(build_board_list(game))
    return boards

def separate_wins(games):
    black_wins = []
    white_wins = []
    
    for game in games:
        if len(game) % 2 == 0:
            black_wins.append(game)
        else:
            white_wins.append(game)
    return (black_wins,white_wins)

def make_boards_for_wins(games):
    (black_wins,white_wins) = separate_wins(games)
    black_board_list = make_boards_for_list(black_wins)
    white_board_list = make_boards_for_list(white_wins)
    return (black_board_list,white_board_list)

In [4]:
def rotate_board(board):
    # Transpose the matrix
    transposed_board = list(zip(*board))
    # Reverse each row to get the 90 degree clockwise rotation
    rotated_board = [list(reversed(row)) for row in transposed_board]
    return rotated_board    

def extend_and_rotate(boards):
    rotated_boards = []
    for board in boards:
        b1 = rotate_board(board)
        b2 = rotate_board(b1)
        b3 = rotate_board(b2)
        rotated_boards.append(board)
        rotated_boards.append(b1)
        rotated_boards.append(b2)
        rotated_boards.append(b3)

    return rotated_boards

In [5]:
#final data doer..
import numpy as np
games = read_all_data()
print ("Loaded {} boards.".format(len(games)))
print(games[0])

[bw,ww] = make_boards_for_wins(games)
X = extend_and_rotate(bw)
y = [1 for _ in range(len(X))]
X.extend(extend_and_rotate(ww))
y.extend([-1 for _ in range(4*len(ww))])

print("{} {}".format(len(X),len(y)))

# Convert lists to NumPy arrays and reshape X to include a channel dimension
X = np.array(X)  # Your list of 19x19 boards
y = np.array(y)  # Your classifications (1 or -1)

X = np.expand_dims(X, -1)  # Add a single channel dimension
print ("Created datasets")


Reading 10496.data
Reading 20108.data
Reading 172889.data
Reading 18546.data
Reading 22400.data
Reading 12022.data
Reading 138931.data
Reading 15965.data
Reading 36878.data
Reading 35242.data
Reading 14572.data
Reading 40525.data
Reading 149404.data
Reading 14529.data
Reading 10131.data
Reading 20440.data
Reading 17637.data
Reading 11956.data
Reading 15411.data
Reading 15283.data
Reading 20322.data
Reading 16197.data
Reading 46121.data
Reading 34947.data
Reading 20115.data
Reading 43712.data
Reading 15076.data
Reading 11936.data
Reading 36979.data
Reading 139373.data
Reading 9400.data
Reading 136053.data
Reading 166925.data
Reading 18574.data
Reading 1704.data
Reading 14003.data
Reading 148181.data
Reading 17341.data
Reading 1728.data
Reading 14096.data
Reading 13704.data
Reading 42635.data
Reading 14239.data
Reading 10407.data
Reading 136249.data
Reading 171218.data
Reading 135957.data
Reading 17458.data
Reading 15609.data
Reading 8831.data
Reading 11944.data
Reading 35959.data
Readin

In [6]:
np.save("X.boards.by.win.loss.npy",X)
np.save("Y.borads.category.npy",y)